# Get summary of crop info for maps

In [ ]:
import os
import sys

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mosaic import *
from ras_tools import *
#from var_dataframe import *

## get summary for # rotations per year
or any single variable previously created with 'ts_composite' tool

In [ ]:
#out_dir = '/home/downspout-cel/paraguay_lc/vector/tests'
#out_path = Path('/home/downspout-cel/paraguay_lc/mosaics')
#cell_list = '/home/downspout-cel/paraguay_lc/mosaics/lists/Training_cells.csv'
#ts_dir = '/home/downspout-cel/paraguay_lc/stac/grids'
#ptfile = '/home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Mar2024_ALL.csv'

#numrot_wet_mosaic = mosaic_cells(cell_list, ts_dir, 'comp/kndvi', 'numrot_wet_2021', out_path)
#numrot_wet_mosaic = '/home/downspout-cel/paraguay_lc/mosaics/numrot_wet_2021_mosaic.tif'
#numrot_yr_mosaic = mosaic_cells(cell_list, ts_dir, 'comp/kndvi', 'numrot_yr_2021', out_path)
#numrot_yr_mosaic = '/home/downspout-cel/paraguay_lc/mosaics/numrot_yr_2021_mosaic.tif'

#samp_numrot_wet = get_variables_at_pts_external2(out_dir, numrot_wet_mosaic, ptfile)

#samp_numrot_yr = pd.read_csv(os.path.join(out_dir,'samp_numrot_yr_2021.csv'))
#samp_numrot_wet = pd.read_csv(os.path.join(out_dir,'samp_numrot_wet_2021.csv'))
#samp_numrot_dry = pd.read_csv(os.path.join(out_dir,'samp_numrot_dry_2021.csv'))
                              
#crop_samp_wet = samp_numrot_wet[(samp_numrot_wet['LC_UNQ'] > 30) & (samp_numrot_wet['LC_UNQ'] < 50)]
#crop_samp_yr = samp_numrot_yr[(samp_numrot_yr['LC_UNQ'] > 30) & (samp_numrot_yr['LC_UNQ'] < 50)]
#crop_samp_wetg = crop_samp_wet[['Class','numrot_wet']].groupby('Class').mean()
#crop_samp_yrg = crop_samp_yr[['Class','numrot_yr']].groupby('Class').mean()
#crop_samp_numrot = crop_samp_yrg.join(crop_samp_wetg,on='Class',how='left')
#print(crop_samp_numrot)

## get map classification for sample

In [ ]:
out_dir = '/home/scratch-cel/cm'
ptfile = '/home/scratch-cel/cm/samp2022_pix.csv'
ras = '/home/downspout-cel/paraguay_lc/mosaics/CEL_base4Poly6_bal300mix3_mosaic.tif'
lut = '/home/scratch-cel/cm/Class_LUT.csv'

check1 = get_variables_at_pts_external(out_dir, ras, ptfile,'map_val','samp_2022_base4Poly6_bal300mix3.csv')

## get crop/no_crop confusion matrix for map

In [ ]:
samp = pd.read_csv(os.path.join(out_dir,'samp_2022_base4Poly6_bal300mix2.csv'))
cm_cropNoCrop = get_confusion_matrix_generic(samp,'map_val','LC_UNQ',lut,'LC2',nodata=0, 
                                     print_cm=False, out_dir=out_dir,model_name='base4Poly6_bal300mix3_2021')
cm_cropNoCrop